# 개요
- 코리빙에 대한 10년치 뉴스를 수집 및 분석
- 광고성 기사는 최대한 배제하고 코리빙에 대한 뉴스 트렌드를 파악하고자함
- clustering을 통해 광고성 기사가 아니면서도 가장 많이 회자되는 뉴스 집단을 2개 선정
- 해당 집단의 LDA를 통해 토픽을 추출 후 트렌드 분석 진행

In [1]:
import pandas as pd

In [2]:
# 경고창 무시
import warnings
warnings.filterwarnings('ignore')

In [3]:
get_news = pd.read_csv('data/project_coliving.news.csv')
get_news

,_id,press,title,content
0,65e80bda8b0bc9bad7e1c213,경인일보,"보증금 떼일 걱정 NO, 개별 호실로 편안하게… 코리빙 ‘디어스판교’",/윤혜경기자hyegyung@kyeongin.com이미 서울지역에선 청년 1인 가구 ...
1,65e80bdc8b0bc9bad7e1c214,강원도민일보,"원주혁신도시 AI 복합시설 건립, VR 스튜디오 선보인다",코리빙 하우스 '커먼타운' 브랜드를 운영 중인 코오롱하우스비전은 오피스텔 일부 호실...
2,65e80bde8b0bc9bad7e1c215,데일리경제,"유니언플레이스, 에이플러스리얼티와 코리빙 및 시니어하우스 업무추진 위한 ...","좌측부터) 에이플러스리얼티 하상관 부사장, 전영하 대표이사, 유니언플레이스 이장호 ..."
3,65e80be08b0bc9bad7e1c216,아시아투데이,"""깡통전세 우려 없어요""…코리빙 하우스 '신촌 맹그로브' 가보니","공유 주거 상품으로 인기 1인 가구 대상… ""사생활 보장"" 보증금 500만원·계약 ..."
4,65e80be28b0bc9bad7e1c217,더벨,"[더벨]에스엘플랫폼, 길동 코리빙하우스 사업 '제자리걸음'",신영그룹 에스엘플랫폼(SLP)과 메테우스자산운용이 추진 중이던 코리빙하우스 사업이 ...
...,...,...,...,...
2154,65e81d128b0bc9bad7e1ca7d,디지털데일리,삼정KPMG ”밀레니얼·Z세대 소비 파워 확대”,"또한, 설계단계에서 세입자의 의사를 반영하여 짓는 ‘퍼즐주택’의 등장과 더불어, 초..."
2155,65e81d158b0bc9bad7e1ca7e,뉴시스,"밀레니얼·Z세대 인구 44% 차지…""노동대체 제품·서비스 관심 높아""","아울러 설계단계에서 세입자의 의사를 반영해 짓는 '퍼즐주택'의 등장과 초소형 주택,..."
2156,65e81d178b0bc9bad7e1ca7f,아시아경제,"삼정KPMG ""밀레니얼·Z세대 소비주도…건조기·가정간편식·홈코노미↑""","설계부터 세입자 의사를 반영해 짓는 '퍼즐주택', 초소형 주택, '코리빙(Co-li..."
2157,65e81d198b0bc9bad7e1ca80,아시아타임즈,"밀레니얼·Z세대 소비파워 확대...""딥리테일 역량 강화해야""","설계부터 세입자 의사를 반영해 짓는 '퍼즐주택', 초소형 주택, '코리빙(Co-li..."


# 데이터 전처리
- 데이터를 살펴본 결과 동일한 내용 또는 매우 비슷한 내용의 기사들이 중복되어 있는것을 확인 (2154,2155 유사기사, 2156,2157 동일기사 등)
- 광고성 뿌리기 기사를 제외하기 위해 content의 내용이 다른 기사와 유사성이 70%이상인 기사들은 동일 기사로 간주하고 drop
- 그 후 title과 content를 합쳐서 하나의 text로 변환
- clustering을 통해 최근 10년간 코리빙에 대한 뉴스를 분석

In [4]:
# 조사, 어미, 구두점 같은 것을 제거
def okt_clean(text):
    from konlpy.tag import Okt
    okt = Okt()
    clean_text = []
    for word in okt.pos(text, stem=True):
        if word[1] not in ['Josa', 'Eomi', 'Punctuation']:
            clean_text.append(word[0])

    return " ".join(clean_text)

In [5]:
get_news['content_trans'] = get_news['content'].apply(okt_clean)

In [6]:
get_news['content_trans'][2100:2120]

2100    로컬 스티치 일반 셰어 하우스 다르다 일 하다 공간 방점 찍히다 있다 코 워킹 코리...
2101    이 다자 요 빈집 수리 하다 게스트하우스 선보이다 것 시작 점차 코 워킹 코리 빙 ...
2102    커먼 타운 은 한국 싱가포르 코리 빙 Co living 사업 펼치다 있다 리베 토 ...
2103    지난 4월 에는 국내 최대 소셜 하우 징 전문 기업 셰어 하우스 우주 인수 하다 셰...
2104    하다 집 방 나누다 쓰다 셰어 하우스 달리 독립 되다 1 인 가구 커뮤니티 시설 결...
2105    코리 빙 120 p Insight 왜 코리 빙 주목 하다 하나 음성 원 도시 건축 ...
2106    커뮤니티 자다 발달 되다 1 인 가구 공유 임대주택 고령화 빠르다 우리 사회 건설 ...
2107    코오롱 글로벌 자회사 코오롱 하우스 비다 작년 12월 서울 강남구 역삼동 고급 공유...
2108    개인 공간 공유 공간 분리 하다 개인 주거 공간 업무 공간 커뮤니티 공간 등 하다 ...
2109    코리 빙 및 코 워킹 주목 받다 국내 최초 프론트 기업 ‘ 로컬 스티치 오다 체인지...
2110    상층 불다 코리 빙 코 워킹 브랜드 로컬 스티치 협업 하다 청년 및 외국인 대상 쉐...
2111    단순하다 ‘ 한국 오다 외국인 이 아니다 장 단기 사이 수 개월 단위 계약 하다 고...
2112    우주 한국 ‘ 코리 빙 트렌드 불러일으키다 기업 하다 2013년 20 여개 셰어 하...
2113    우주 한국 코리 빙 트렌드 불러일으키다 기업 하다 2013년 20 여개 셰어 하우스...
2114    우주 한국 ‘ 코리 빙 트렌드 불러일으키다 기업 하다 2013년 20 여개 셰어 하...
2115    우주 한국 코리 빙 트렌드 불러일으키다 기업 2013년 20 여개 셰어 하우스 임대...
2116    우주 한국 ‘ 코리 빙 트렌드 불러일으키다 기업 하다 2013년 20 여개 셰어 하...
2117    우주 한국 

In [7]:
# 문장 정리 및 재정제
def solve_text(text):
    change_dict = {
        '초소 형': '초소형',
        '신 영 그룹' : '신영그룹',
        '세 입자' : '세입자',
        '젠틀 몬스터' : '젠틀몬스터',
        '하우스 도산' : '하우스도산',
        '코 워킹' : '코워킹',
        '드림 하우스' : '드림하우스',
        '홈즈 컴퍼니' : '홈즈컴퍼니',
        '기업 형' : '기업형',
        '홈즈 스튜디오' : '홈즈스튜디오',
        '디 어스' : '디어스',
        '현 대' : '현대',
        '오 일 뱅크' : '오일뱅크',
        '셰어 하우스' : '쉐어하우스',
        '쉐어 하우스' : '쉐어하우스',
        '워커 스 하이' : '워커스하이',
        '엠 알 브이' : '엠알브이',
        '임대 형' : '임대형',
        '코리...' : '코리빙',
        '...':'',
        '코리 빙' : '코리빙',
        "'" : '',
        '"' : '',
        '1 인': '1인',
        '커먼 타운' : '커먼타운',
        '퍼스 널' : '퍼스널',
        '구 입시' : '구입',
        'co living house' : 'co-living-house',
        '에스테 이' : '에스테이',
        '야 놀다' : '야놀자',
        '트러스 테이' : '트러스테이',
        '이용 료' : '이용료',
        '위워 크다' : '위워크',
        'co living' : 'co-living',
        '낙성' : '낙성대',
        '베다 처 창업 센터' : '벤처창업센터',
        '코리빙 하우스' : '코리빙하우스'
    }
    for i in change_dict:
        text = text.replace(i,change_dict[i])
    return text

In [8]:
'''
def to_list(content):
    return list(content.split())
    
def equality_check(get_list):
    isSamedict = {k:False for k in range(len(get_list))}
    for i in range(len(get_list)):
        for j in range(i+1,len(get_list)):
            if not isSamedict[j]:
                set1 = set(get_list[i])
                set2 = set(get_list[j])
                
                total_items = len(set1) + len(set2)
                overlap_count = len(set1 & set2)
                overlap_probability = (overlap_count / total_items)*2
    
                if overlap_probability >= 0.6:
                    isSamedict[j] = True

    return list(isSamedict.values())
'''

'\ndef to_list(content):\n    return list(content.split())\n    \ndef equality_check(get_list):\n    isSamedict = {k:False for k in range(len(get_list))}\n    for i in range(len(get_list)):\n        for j in range(i+1,len(get_list)):\n            if not isSamedict[j]:\n                set1 = set(get_list[i])\n                set2 = set(get_list[j])\n                \n                total_items = len(set1) + len(set2)\n                overlap_count = len(set1 & set2)\n                overlap_probability = (overlap_count / total_items)*2\n    \n                if overlap_probability >= 0.6:\n                    isSamedict[j] = True\n\n    return list(isSamedict.values())\n'

In [9]:
get_news['content_trans'] = get_news['content_trans'].apply(solve_text)
# get_news['content_trans'] = get_news['content_trans'].apply(to_list)

In [10]:
# get_news['content_trans_same'] = equality_check(get_news['content_trans'])

## 벡터화

In [11]:
# tokenizing
from sklearn.feature_extraction.text import TfidfVectorizer
tfidVectorizer = TfidfVectorizer()

In [12]:
features = tfidVectorizer.fit_transform(get_news['content_trans'])

## 유사도 확인

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

def equality_check_cosine(get_list, features):
    features_array = features.toarray()
    
    isSamedict = {k: False for k in range(len(get_list))}
    
    for i in range(len(get_list)):
        for j in range(i + 1, len(get_list)):
            if not isSamedict[j]:
                similarity = cosine_similarity(features_array[i:i+1], features_array[j:j+1])[0][0]
                if similarity >= 0.7:
                    isSamedict[j] = True
                    
    return list(isSamedict.values())

In [14]:
get_news['content_trans_same'] = equality_check_cosine(get_news['content_trans'],features)

In [15]:
get_news

,_id,press,title,content,content_trans,content_trans_same
0,65e80bda8b0bc9bad7e1c213,경인일보,"보증금 떼일 걱정 NO, 개별 호실로 편안하게… 코리빙 ‘디어스판교’",/윤혜경기자hyegyung@kyeongin.com이미 서울지역에선 청년 1인 가구 ...,윤혜경 기자 hyegyung@kyeongin.com 이미 서 울 지역 청년 1인 가...,False
1,65e80bdc8b0bc9bad7e1c214,강원도민일보,"원주혁신도시 AI 복합시설 건립, VR 스튜디오 선보인다",코리빙 하우스 '커먼타운' 브랜드를 운영 중인 코오롱하우스비전은 오피스텔 일부 호실...,코리빙하우스 커먼타운 브랜드 운영 중인 코오롱 하우스 비다 오피스텔 일부 호실 위탁...,False
2,65e80bde8b0bc9bad7e1c215,데일리경제,"유니언플레이스, 에이플러스리얼티와 코리빙 및 시니어하우스 업무추진 위한 ...","좌측부터) 에이플러스리얼티 하상관 부사장, 전영하 대표이사, 유니언플레이스 이장호 ...",좌측 에이 플러스 리얼 티 하 상관 부사 장 전 영하 대표이사 유니 언플 레이스 이...,False
3,65e80be08b0bc9bad7e1c216,아시아투데이,"""깡통전세 우려 없어요""…코리빙 하우스 '신촌 맹그로브' 가보니","공유 주거 상품으로 인기 1인 가구 대상… ""사생활 보장"" 보증금 500만원·계약 ...",공유 주거 상품 인기 1인 가구 대상 사생활 보장 보증금 500만원 계약 6 개월 ...,False
4,65e80be28b0bc9bad7e1c217,더벨,"[더벨]에스엘플랫폼, 길동 코리빙하우스 사업 '제자리걸음'",신영그룹 에스엘플랫폼(SLP)과 메테우스자산운용이 추진 중이던 코리빙하우스 사업이 ...,신영그룹 에스엘 플랫폼 SLP 과 메다 우스 자산운용 추진 중 코리빙하우스 사업 제...,False
...,...,...,...,...,...,...
2154,65e81d128b0bc9bad7e1ca7d,디지털데일리,삼정KPMG ”밀레니얼·Z세대 소비 파워 확대”,"또한, 설계단계에서 세입자의 의사를 반영하여 짓는 ‘퍼즐주택’의 등장과 더불어, 초...",또한 설계 단계 세입자 의사 반영 하다 짓다 ‘ 퍼즐 주택 의 등장 더불다 초소형 ...,True
2155,65e81d158b0bc9bad7e1ca7e,뉴시스,"밀레니얼·Z세대 인구 44% 차지…""노동대체 제품·서비스 관심 높아""","아울러 설계단계에서 세입자의 의사를 반영해 짓는 '퍼즐주택'의 등장과 초소형 주택,...",아우르다 설계 단계 세입자 의사 반영 하다 짓다 퍼즐 주택 의 등장 초소형 주택 개...,True
2156,65e81d178b0bc9bad7e1ca7f,아시아경제,"삼정KPMG ""밀레니얼·Z세대 소비주도…건조기·가정간편식·홈코노미↑""","설계부터 세입자 의사를 반영해 짓는 '퍼즐주택', 초소형 주택, '코리빙(Co-li...",설계 세입자 의사 반영 하다 짓다 퍼즐 주택 초소형 주택 코리빙 Co living ...,False
2157,65e81d198b0bc9bad7e1ca80,아시아타임즈,"밀레니얼·Z세대 소비파워 확대...""딥리테일 역량 강화해야""","설계부터 세입자 의사를 반영해 짓는 '퍼즐주택', 초소형 주택, '코리빙(Co-li...",설계 세입자 의사 반영 하다 짓다 퍼즐 주택 초소형 주택 코리빙 Co living ...,True


In [16]:
df_unique_news = get_news.query('content_trans_same == False')

In [17]:
df_unique_news

,_id,press,title,content,content_trans,content_trans_same
0,65e80bda8b0bc9bad7e1c213,경인일보,"보증금 떼일 걱정 NO, 개별 호실로 편안하게… 코리빙 ‘디어스판교’",/윤혜경기자hyegyung@kyeongin.com이미 서울지역에선 청년 1인 가구 ...,윤혜경 기자 hyegyung@kyeongin.com 이미 서 울 지역 청년 1인 가...,False
1,65e80bdc8b0bc9bad7e1c214,강원도민일보,"원주혁신도시 AI 복합시설 건립, VR 스튜디오 선보인다",코리빙 하우스 '커먼타운' 브랜드를 운영 중인 코오롱하우스비전은 오피스텔 일부 호실...,코리빙하우스 커먼타운 브랜드 운영 중인 코오롱 하우스 비다 오피스텔 일부 호실 위탁...,False
2,65e80bde8b0bc9bad7e1c215,데일리경제,"유니언플레이스, 에이플러스리얼티와 코리빙 및 시니어하우스 업무추진 위한 ...","좌측부터) 에이플러스리얼티 하상관 부사장, 전영하 대표이사, 유니언플레이스 이장호 ...",좌측 에이 플러스 리얼 티 하 상관 부사 장 전 영하 대표이사 유니 언플 레이스 이...,False
3,65e80be08b0bc9bad7e1c216,아시아투데이,"""깡통전세 우려 없어요""…코리빙 하우스 '신촌 맹그로브' 가보니","공유 주거 상품으로 인기 1인 가구 대상… ""사생활 보장"" 보증금 500만원·계약 ...",공유 주거 상품 인기 1인 가구 대상 사생활 보장 보증금 500만원 계약 6 개월 ...,False
4,65e80be28b0bc9bad7e1c217,더벨,"[더벨]에스엘플랫폼, 길동 코리빙하우스 사업 '제자리걸음'",신영그룹 에스엘플랫폼(SLP)과 메테우스자산운용이 추진 중이던 코리빙하우스 사업이 ...,신영그룹 에스엘 플랫폼 SLP 과 메다 우스 자산운용 추진 중 코리빙하우스 사업 제...,False
...,...,...,...,...,...,...
2147,65e81d048b0bc9bad7e1ca76,뉴스인사이드,[잡인사이드] 클리니어 상용직 채용,[직무내용] * 코리빙스페이스(대부분 여성 전용)를 주기적으로 방문하여 현장 관리 ...,직무 내용 코리빙 스페이스 대부분 여성 전용 를 주 기적 방문 하다 현장 관리 상태...,False
2148,65e81d068b0bc9bad7e1ca77,업다운뉴스,"밀레니얼‧Z세대가 새 소비트렌드 주역, 신소비시대의 기업에 필요한 '5대 DN...","특히 설계단계부터 세입자의 뜻을 담아내 건설하는 '퍼즐주택'의 등장과 초소형 주택,...",특히 설계 단계 세입자 뜻 담다 건설 하다 퍼즐 주택 의 등장 초소형 주택 개인 라...,False
2149,65e81d088b0bc9bad7e1ca78,헤럴드경제,삼정KPMG ”밀레니얼ㆍZ세대 소비력 확대…사업 ‘리포지션’ 필요”,"또한 설계단계에서 세입자의 의사를 반영해 짓는 ‘퍼즐주택’, 개인의 생활방식을 고수...",또한 설계 단계 세입자 의사 반영 하다 짓다 ‘ 퍼즐 주택 개인 생활 방식 고수 하...,False
2156,65e81d178b0bc9bad7e1ca7f,아시아경제,"삼정KPMG ""밀레니얼·Z세대 소비주도…건조기·가정간편식·홈코노미↑""","설계부터 세입자 의사를 반영해 짓는 '퍼즐주택', 초소형 주택, '코리빙(Co-li...",설계 세입자 의사 반영 하다 짓다 퍼즐 주택 초소형 주택 코리빙 Co living ...,False


In [18]:
df_unique_news['title_trans'] = df_unique_news['title'].apply(okt_clean)
df_unique_news['title_trans'] = df_unique_news['title_trans'].apply(solve_text)

In [19]:
df_unique_news = df_unique_news.drop(columns=['content_trans_same','press','_id'])

In [20]:
df_unique_news.columns

Index(['title', 'content', 'content_trans', 'title_trans'], dtype='object')

In [21]:
def to_total(columns_list, dataframe):
    final_list = []
    for index, row in dataframe.iterrows():
        total_text = ''
        for col in columns_list:
            total_text += str(row[col])
            total_text += ' '
        final_list.append(total_text.strip())
    return final_list

In [22]:
what_sum = ['title_trans','content_trans']
df_unique_news['total'] =  to_total(what_sum,df_unique_news)

In [23]:
df_unique_news

,title,content,content_trans,title_trans,total
0,"보증금 떼일 걱정 NO, 개별 호실로 편안하게… 코리빙 ‘디어스판교’",/윤혜경기자hyegyung@kyeongin.com이미 서울지역에선 청년 1인 가구 ...,윤혜경 기자 hyegyung@kyeongin.com 이미 서 울 지역 청년 1인 가...,보증금 떼이다 걱정 NO 개별 호실 편안하다 코리빙 ‘ 디어스 판교,보증금 떼이다 걱정 NO 개별 호실 편안하다 코리빙 ‘ 디어스 판교 윤혜경 기자 h...
1,"원주혁신도시 AI 복합시설 건립, VR 스튜디오 선보인다",코리빙 하우스 '커먼타운' 브랜드를 운영 중인 코오롱하우스비전은 오피스텔 일부 호실...,코리빙하우스 커먼타운 브랜드 운영 중인 코오롱 하우스 비다 오피스텔 일부 호실 위탁...,원주 혁신도시 AI 복합 시설 건립 VR 스튜디오 선보이다,원주 혁신도시 AI 복합 시설 건립 VR 스튜디오 선보이다 코리빙하우스 커먼타운 브...
2,"유니언플레이스, 에이플러스리얼티와 코리빙 및 시니어하우스 업무추진 위한 ...","좌측부터) 에이플러스리얼티 하상관 부사장, 전영하 대표이사, 유니언플레이스 이장호 ...",좌측 에이 플러스 리얼 티 하 상관 부사 장 전 영하 대표이사 유니 언플 레이스 이...,유니 언플 레이스 에이 플러스 리얼 티 코리빙 및 시니어 하우스 업무 추진 위,유니 언플 레이스 에이 플러스 리얼 티 코리빙 및 시니어 하우스 업무 추진 위 좌측...
3,"""깡통전세 우려 없어요""…코리빙 하우스 '신촌 맹그로브' 가보니","공유 주거 상품으로 인기 1인 가구 대상… ""사생활 보장"" 보증금 500만원·계약 ...",공유 주거 상품 인기 1인 가구 대상 사생활 보장 보증금 500만원 계약 6 개월 ...,깡통 전세 우려 없다 코리빙하우스 신촌 맹그로브 가보다,깡통 전세 우려 없다 코리빙하우스 신촌 맹그로브 가보다 공유 주거 상품 인기 1인 ...
4,"[더벨]에스엘플랫폼, 길동 코리빙하우스 사업 '제자리걸음'",신영그룹 에스엘플랫폼(SLP)과 메테우스자산운용이 추진 중이던 코리빙하우스 사업이 ...,신영그룹 에스엘 플랫폼 SLP 과 메다 우스 자산운용 추진 중 코리빙하우스 사업 제...,더벨 에스엘 플랫폼 길동 코리빙하우스 사업 제자리걸음,더벨 에스엘 플랫폼 길동 코리빙하우스 사업 제자리걸음 신영그룹 에스엘 플랫폼 SLP...
...,...,...,...,...,...
2147,[잡인사이드] 클리니어 상용직 채용,[직무내용] * 코리빙스페이스(대부분 여성 전용)를 주기적으로 방문하여 현장 관리 ...,직무 내용 코리빙 스페이스 대부분 여성 전용 를 주 기적 방문 하다 현장 관리 상태...,잡 사이드 클리 니 어 상용 직 채용,잡 사이드 클리 니 어 상용 직 채용 직무 내용 코리빙 스페이스 대부분 여성 전용 ...
2148,"밀레니얼‧Z세대가 새 소비트렌드 주역, 신소비시대의 기업에 필요한 '5대 DN...","특히 설계단계부터 세입자의 뜻을 담아내 건설하는 '퍼즐주택'의 등장과 초소형 주택,...",특히 설계 단계 세입자 뜻 담다 건설 하다 퍼즐 주택 의 등장 초소형 주택 개인 라...,밀레 얼 ‧ Z 세대 새 소비 트렌드 주역 신 소 비시 대 기업 필요하다 5 대다 DN,밀레 얼 ‧ Z 세대 새 소비 트렌드 주역 신 소 비시 대 기업 필요하다 5 대다 ...
2149,삼정KPMG ”밀레니얼ㆍZ세대 소비력 확대…사업 ‘리포지션’ 필요”,"또한 설계단계에서 세입자의 의사를 반영해 짓는 ‘퍼즐주택’, 개인의 생활방식을 고수...",또한 설계 단계 세입자 의사 반영 하다 짓다 ‘ 퍼즐 주택 개인 생활 방식 고수 하...,삼정 KPMG ” 밀레 얼 ㆍ Z 세대 소비 력 확대 사업 ‘ 리 포지션 필요 ”,삼정 KPMG ” 밀레 얼 ㆍ Z 세대 소비 력 확대 사업 ‘ 리 포지션 필요 ” ...
2156,"삼정KPMG ""밀레니얼·Z세대 소비주도…건조기·가정간편식·홈코노미↑""","설계부터 세입자 의사를 반영해 짓는 '퍼즐주택', 초소형 주택, '코리빙(Co-li...",설계 세입자 의사 반영 하다 짓다 퍼즐 주택 초소형 주택 코리빙 Co living ...,삼정 KPMG 밀레 얼 Z 세대 소비 주도 건조기 가정 간편식 홈코 노미 ↑,삼정 KPMG 밀레 얼 Z 세대 소비 주도 건조기 가정 간편식 홈코 노미 ↑ 설계 ...


## tfidf 및 시각화

In [24]:
import pyLDAvis
import pyLDAvis.lda_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [25]:
tfidfVectorizer = TfidfVectorizer(max_df=0.95, min_df=2)

In [26]:
tfidf_matrix = tfidfVectorizer.fit_transform(df_unique_news['total'])

### clusturing

In [27]:
from sklearn.cluster import KMeans

#### 최적 k값 찾기
- silhouette사용

In [28]:
from sklearn.metrics import silhouette_score

In [29]:
k_range = range(2,30)

best_n = -1
best_score = -1

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(tfidf_matrix)
    clusters = kmeans.predict(tfidf_matrix)

    score = silhouette_score(tfidf_matrix,clusters)
    print('k :',k,'score :',score)

    if score > best_score:
        best_n = k
        best_score = score

print('best n :', best_n, 'best score :', best_score)

k : 2 score : 0.03203376979172018
k : 3 score : 0.028086912514146013
k : 4 score : 0.031222601895904322
k : 5 score : 0.032380262631783376
k : 6 score : 0.036291286907389225
k : 7 score : 0.03671022988877178
k : 8 score : 0.03831592071235061
k : 9 score : 0.038690670047760174
k : 10 score : 0.03923109965081779
k : 11 score : 0.03838925178495297
k : 12 score : 0.03841352440742459
k : 13 score : 0.037916103369797095
k : 14 score : 0.03942789976296405
k : 15 score : 0.04134519876358404
k : 16 score : 0.042054162429129606
k : 17 score : 0.021824934115614144
k : 18 score : 0.022401529970547365
k : 19 score : 0.021678537794199988
k : 20 score : 0.021519698715809783
k : 21 score : 0.020568206227638792
k : 22 score : 0.020402695300878695
k : 23 score : 0.018281305482677004
k : 24 score : 0.01853233767274944
k : 25 score : 0.01777132322377559
k : 26 score : 0.018270783646793128
k : 27 score : 0.019615606532310377
k : 28 score : 0.01996789206255391
k : 29 score : 0.019792242132241032
best n : 16

- 실루엣 테스트 결과 최적 k 값 18

In [31]:
kmeans = KMeans(n_clusters=16, random_state=42)
kmeans.fit(tfidf_matrix)

KMeans(n_clusters=16, random_state=42)

In [32]:
len(tfidf_matrix.toarray())

906

In [33]:
cluster_labels = kmeans.labels_

In [34]:
df_unique_news['kmeans_labels'] = cluster_labels

## 군집별 해석
- 가장 많은 군집을 이루고 있는 상위 2개 집단 해석

In [35]:
# 상위 집단 찾기
df_unique_news['kmeans_labels'].value_counts()

kmeans_labels
13    115
15    109
5     100
11     87
3      82
2      73
4      68
9      67
6      37
10     31
14     28
8      26
0      24
1      23
7      20
12     16
Name: count, dtype: int64

- 13, 15번 군집이 가장 많은 카운트를 보유

In [36]:
df_top_group = df_unique_news.query('kmeans_labels == 13 or kmeans_labels == 15')
df_top_group

,title,content,content_trans,title_trans,total,kmeans_labels
2,"유니언플레이스, 에이플러스리얼티와 코리빙 및 시니어하우스 업무추진 위한 ...","좌측부터) 에이플러스리얼티 하상관 부사장, 전영하 대표이사, 유니언플레이스 이장호 ...",좌측 에이 플러스 리얼 티 하 상관 부사 장 전 영하 대표이사 유니 언플 레이스 이...,유니 언플 레이스 에이 플러스 리얼 티 코리빙 및 시니어 하우스 업무 추진 위,유니 언플 레이스 에이 플러스 리얼 티 코리빙 및 시니어 하우스 업무 추진 위 좌측...,13
3,"""깡통전세 우려 없어요""…코리빙 하우스 '신촌 맹그로브' 가보니","공유 주거 상품으로 인기 1인 가구 대상… ""사생활 보장"" 보증금 500만원·계약 ...",공유 주거 상품 인기 1인 가구 대상 사생활 보장 보증금 500만원 계약 6 개월 ...,깡통 전세 우려 없다 코리빙하우스 신촌 맹그로브 가보다,깡통 전세 우려 없다 코리빙하우스 신촌 맹그로브 가보다 공유 주거 상품 인기 1인 ...,15
11,"오피스텔 수익률, GBDYBDCBD보다 G밸리…주거 대안으로 ‘코리빙하우스’ 부상",오피스텔 월세가 상승하면서 새로운 주거 대안으로 ‘코리빙하우스’가 떠오르고 있다. ...,오피스텔 월세 상승 하다 새롭다 주거 대안 ‘ 코리빙하우스 가다 떠오르다 있다 ‘ ...,오피스텔 수익률 GBDYBDCBD 보다 G 밸리 주거 대안 ‘ 코리빙하우스 부상,오피스텔 수익률 GBDYBDCBD 보다 G 밸리 주거 대안 ‘ 코리빙하우스 부상 오...,15
12,입주자 만족도 높은 코리빙하우스 ‘가산 셀립’,최근 2030대 직장인들이 늘어나면서 공유주거의 일종인 ‘코리빙하우스’가 주목받고 ...,최근 2030 대다 직장인 들 늘어나다 공유 주거 일종 ‘ 코리빙하우스 가다 주목 ...,입 주자 만족도 높다 코리빙하우스 ‘ 가산 셀립,입 주자 만족도 높다 코리빙하우스 ‘ 가산 셀립 최근 2030 대다 직장인 들 늘어...,15
23,1인가구 대안집 '코리빙 하우스' 차별화 전략 통했나,'코리빙 하우스'는 한 건물에 개인이 사는 공간이 있고 여럿이 함께 쓰는 공간이 있...,코리빙하우스 늘다 하다 건물 개인 살다 공간 있다 여럿 함께 쓰다 공간 있다 공유 ...,1인 가구 대 안집 코리빙하우스 차별 화 전략 통 하다,1인 가구 대 안집 코리빙하우스 차별 화 전략 통 하다 코리빙하우스 늘다 하다 건물...,15
...,...,...,...,...,...,...
2137,"[더벨]'자금 조달' 로컬스티치, 공유건물 확충 박차",로컬스티치는 조달한 자금으로 공유 사무실과 셰어하우스를 결합한 '코워킹(co-wor...,로컬 스티치 조달 자금 공유 사무실 쉐어하우스 결합 코워킹 co working 코리...,더벨 자금 조달 로컬 스티치 공유 건물 확충 박차,더벨 자금 조달 로컬 스티치 공유 건물 확충 박차 로컬 스티치 조달 자금 공유 사무...,13
2138,집다운 집을 선택하는 특별한 기준,코오롱 커먼타운의 공유주택 ‘트리하우스’를 설계한 건축가 송멜로디는 공용 공간을 이...,코오롱 커먼타운 공유 주택 ‘ 트리하우스 를 설계 건축가 송 멜로디 공용 공간 이웃...,집다 집 선택 하다 특별하다 기준,집다 집 선택 하다 특별하다 기준 코오롱 커먼타운 공유 주택 ‘ 트리하우스 를 설계...,15
2143,당산역 인근 복합생활공간 '유니언타운' 오픈,창업을 준비하는 청년들을 위한 공유주방 넥스트키친과 코워킹 스페이스를 기획했다. 상...,창업 준비 하다 청년 들 위 공유 주방 넥스트 키친 코워킹 스페이스 기획 하다 상층...,당산역 인근 복합 생활 공간 유니언타운 오픈,당산역 인근 복합 생활 공간 유니언타운 오픈 창업 준비 하다 청년 들 위 공유 주방...,13
2147,[잡인사이드] 클리니어 상용직 채용,[직무내용] * 코리빙스페이스(대부분 여성 전용)를 주기적으로 방문하여 현장 관리 ...,직무 내용 코리빙 스페이스 대부분 여성 전용 를 주 기적 방문 하다 현장 관리 상태...,잡 사이드 클리 니 어 상용 직 채용,잡 사이드 클리 니 어 상용 직 채용 직무 내용 코리빙 스페이스 대부분 여성 전용 ...,13


### 토픽 추출 (그룹 15)

In [37]:
vectorizer = TfidfVectorizer(use_idf=True)
features = vectorizer.fit_transform(df_top_group.query('kmeans_labels == 15')['total'])

In [38]:
lda_model = LatentDirichletAllocation(n_components=5, random_state=42)
lda_model.fit(features)

LatentDirichletAllocation(n_components=5, random_state=42)

In [39]:
dictionary_list = vectorizer.get_feature_names_out()

In [40]:
topics_output = lda_model.transform(features)
df_topics_score = pd.DataFrame(data=topics_output)

In [41]:
df_topics_score

,0,1,2,3,4
0,0.885499,0.028486,0.029015,0.028368,0.028633
1,0.202517,0.699521,0.033059,0.032049,0.032854
2,0.028951,0.028162,0.028385,0.027829,0.886673
3,0.030658,0.880322,0.030064,0.029203,0.029752
4,0.879315,0.030132,0.030354,0.029917,0.030282
...,...,...,...,...,...
104,0.026970,0.891973,0.027040,0.026870,0.027146
105,0.029191,0.028995,0.883545,0.028870,0.029398
106,0.029238,0.028981,0.029762,0.028726,0.883293
107,0.031049,0.029733,0.879522,0.029557,0.030139


In [42]:
import numpy as np

In [43]:
df_topics_score['dominant_topic_number'] = np.argmax(topics_output, axis=1)

#### 토픽 주제 확인

In [44]:
## 상위 단어 추출
## 0 확률 1은 dictionary
topics_list = list()
for topic in lda_model.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topics[:3])
    topics_text = ' '.join(df_topics[1].values[:10])# 시리즈 형식으로 출력 get values from series / index
    print(topics_text)
    topics_list.append(topics_text)


topics_list_add = [topics_list, ['Topic0', 'Topic1', 'Topic2','Topic3','Topic4']]
df_topics_keywords = pd.DataFrame(topics_list_add)

하다 코리빙하우스 공유 주거 쉐어하우스 공간 형태 있다 주택 코리빙
하다 있다 공유 주거 쉐어하우스 공간 코리빙하우스 살다 living 오피스텔
하다 주거 공유 공간 쉐어하우스 코리빙 1인 있다 혼자 코리빙하우스
연애 귀찮다 세계 김소은 라이브러리 충분하다 지현우 달리 쓰다 현장
공유 하다 주거 공간 코리빙 되다 코리빙하우스 세대 하우스 있다


In [45]:
df_topics_keywords

,0,1,2,3,4
0,하다 코리빙하우스 공유 주거 쉐어하우스 공간 형태 있다 주택 코리빙,하다 있다 공유 주거 쉐어하우스 공간 코리빙하우스 살다 living 오피스텔,하다 주거 공유 공간 쉐어하우스 코리빙 1인 있다 혼자 코리빙하우스,연애 귀찮다 세계 김소은 라이브러리 충분하다 지현우 달리 쓰다 현장,공유 하다 주거 공간 코리빙 되다 코리빙하우스 세대 하우스 있다
1,Topic0,Topic1,Topic2,Topic3,Topic4


### 토픽 추출 (그룹 13)

In [46]:
vectorizer = TfidfVectorizer(use_idf=True)
features = vectorizer.fit_transform(df_top_group.query('kmeans_labels == 13')['total'])

In [47]:
lda_model = LatentDirichletAllocation(n_components=5, random_state=42)
lda_model.fit(features)

LatentDirichletAllocation(n_components=5, random_state=42)

In [48]:
dictionary_list = vectorizer.get_feature_names_out()

In [49]:
topics_output = lda_model.transform(features)
df_topics_score = pd.DataFrame(data=topics_output)

In [50]:
df_topics_score

,0,1,2,3,4
0,0.032525,0.032932,0.869409,0.032610,0.032524
1,0.887056,0.028279,0.028214,0.028177,0.028274
2,0.028869,0.884609,0.028772,0.028716,0.029034
3,0.030530,0.876898,0.030720,0.031259,0.030593
4,0.028056,0.028859,0.028314,0.886490,0.028281
...,...,...,...,...,...
110,0.889558,0.027473,0.027650,0.027511,0.027807
111,0.029707,0.881839,0.029448,0.029419,0.029587
112,0.027961,0.888767,0.027969,0.027666,0.027637
113,0.876040,0.031107,0.031005,0.030903,0.030945


In [51]:
import numpy as np

In [52]:
df_topics_score['dominant_topic_number'] = np.argmax(topics_output, axis=1)

#### 토픽 주제 확인

In [53]:
## 상위 단어 추출
## 0 확률 1은 dictionary
topics_list = list()
for topic in lda_model.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topics[:3])
    topics_text = ' '.join(df_topics[1].values[:10])# 시리즈 형식으로 출력 get values from series / index
    print(topics_text)
    topics_list.append(topics_text)


topics_list_add = [topics_list, ['Topic0', 'Topic1', 'Topic2','Topic3','Topic4']]
df_topics_keywords = pd.DataFrame(topics_list_add)

하다 있다 공간 스티치 로컬 코워킹 코리빙하우스 에어비앤비 지역 공유
하다 공간 되다 코리빙 공유 코리빙하우스 있다 개인 호텔 주거
하다 공간 있다 비상 주거 로컬 코워킹 크다 스티치 코리빙하우스
하다 되다 코리빙하우스 있다 공간 주거 주방 홈즈 공유 리빙
하다 공간 있다 되다 형태 쉐어하우스 코리빙하우스 집무실 개인 문화


In [54]:
df_topics_keywords

,0,1,2,3,4
0,하다 있다 공간 스티치 로컬 코워킹 코리빙하우스 에어비앤비 지역 공유,하다 공간 되다 코리빙 공유 코리빙하우스 있다 개인 호텔 주거,하다 공간 있다 비상 주거 로컬 코워킹 크다 스티치 코리빙하우스,하다 되다 코리빙하우스 있다 공간 주거 주방 홈즈 공유 리빙,하다 공간 있다 되다 형태 쉐어하우스 코리빙하우스 집무실 개인 문화
1,Topic0,Topic1,Topic2,Topic3,Topic4
